In [15]:
'''
    this is the final polynomial regression models I am builing to get the required results
'''

############################################# Necessary dependancies #############################################
import talib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

############################################ Class and helper functions ###########################################

    
class PolynomialRegressionMultiOutput:
     
    def __init__(self, degree, learning_rate, iterations, batch_size=None):
        self.degree = degree
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.batch_size = batch_size
        self.W = None
        self.mean = None
        self.std = None

    # Function to transform X into polynomial features
    def transform(self, X):
        # Start with the bias term (X^0 = 1)
        X_transform = np.ones((X.shape[0], 1))  # Shape: (n_samples, 1)
        
        # Loop through degrees from 1 to the specified degree
        for j in range(1, self.degree + 1):
            for feature in range(X.shape[1]):  # Loop through all features (columns) in X
                X_pow = np.power(X[:, feature], j).reshape(-1, 1)  # Raise each feature to the power j
                X_transform = np.hstack((X_transform, X_pow))  # Stack the new column horizontally
            
        return X_transform

    # Function to normalize the features (save mean and std for normalization)
    def normalize(self, X):
        if self.mean is None or self.std is None:
            self.mean = np.mean(X[:, 1:], axis=0)  # Mean excluding the bias term
            self.std = np.std(X[:, 1:], axis=0)    # Standard deviation excluding the bias term
        X[:, 1:] = (X[:, 1:] - self.mean) / self.std
        return X

    # Model training using mini-batch gradient descent
    def fit(self, X, Y):
        self.X = X
        self.Y = Y
        self.m, self.n = self.X.shape
        self.n_outputs = self.Y.shape[1]  # Number of output variables

        # Transform X into polynomial features
        X_transform = self.transform(self.X)

        # Normalize the transformed features
        X_normalize = self.normalize(X_transform)

        # Initialize weights for multi-output (n_features, n_outputs)
        self.W = np.zeros((X_normalize.shape[1], self.n_outputs))

        # print(f'model training: x_normalize shape:{X_normalize.shape}\nWeights shape: {self.W.shape}')
        
        # If no batch size is set, process the entire dataset at once (default to full-batch gradient descent)
        if self.batch_size is None:
            self.batch_size = self.m
        
        # Gradient descent learning with mini-batch training
        for i in range(self.iterations):
            # Iterate over mini-batches
            for start in range(0, self.m, self.batch_size):
                end = min(start + self.batch_size, self.m)
                X_batch = X_normalize[start:end]
                Y_batch = self.Y[start:end]

                # Compute predictions for the mini-batch
                h = np.dot(X_batch, self.W)

                # Calculate error
                error = h - Y_batch

                # Update weights
                self.W = self.W - self.learning_rate * (1 / X_batch.shape[0]) * np.dot(X_batch.T, error)

                # if i == 0 and start == 0:
                    # print(f'Initial batch error shape: {error.shape}, X_batch shape: {X_batch.shape}')

        return self

    # Predict function using the learned weights
    def predict(self, X):
        # Transform and normalize X for prediction
        X_transform = self.transform(X)
        X_normalize = self.normalize(X_transform)

        # Ensure output is a matrix of shape (n_samples, n_outputs)
        return np.dot(X_normalize, self.W)

def multivariateFeatureEngineering(data):
    
    #Trend following Indicators:

    #SMA - identofy long term trend
    data['50_sma'] = data['Close'].rolling(window=50).mean() 
    data['200_sma'] = data['Close'].rolling(window=200).mean() 

    #EMA - trend analysis: more weight applied to recent points
    data['50_ema'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['100_ema'] = data['Close'].ewm(span=100, adjust=False).mean()

    #MACD
    data['12_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['26_ema'] = data['Close'].ewm(span=26, adjust=False).mean()

    data['MACD_line'] = data['12_ema']-data['26_ema'] # calculate the MACD line
    data['Signal_line'] = data['MACD_line'].ewm(span=9, adjust=False).mean() # 9-preiod ema signal calculated from the Macdline
    # data['MACD_histogram'] = data['MACD_line'] - data['Signal_line']

    #ADX
    # Calculate ADX using TA-Lib (14-period by default)
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)

    #Momentum indicators:

    #RSI - 14-period
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
    
    #Stochastic Oscillator
    data['stoch_k'], data['stoch_d'] = talib.STOCH(data['High'], data['Low'], data['Close'], fastk_period=14, slowk_period=3, slowd_period=3)

    #Volatility indicators#:

    #ATR -Default period for ATR is 14
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

    data = data.dropna() # drop rows that have NA

    #drop certain featires
    data = data.drop(columns=['12_ema', '26_ema'])

    return data

def multivariateFeatureLagMultiStep(data, n_past, future_steps, target_column=1):
    features = []
    response = []

    max_future_step = max(future_steps)
    num_features = data.shape[1]
    group_feature_lags =  1 # change grouping of lagged features

    # Adjust the loop to prevent index out of bounds
    for i in range(n_past, len(data) - max_future_step + 1):

        if group_feature_lags==1:
                
            lagged_features = []

            for feature_idx in range(num_features):
                feature_lags = data.iloc[i - n_past:i, feature_idx].values 
                lagged_features.extend(feature_lags) 

        elif group_feature_lags==0:
            features.append(data.iloc[i - n_past:i, :].values)  # Take all columns as features

        # Use .iloc for integer-based indexing and .values to get a NumPy array

        if group_feature_lags==1:
            features.append(lagged_features)

        # Extract the target values at specified future steps using .iloc
        response.append([data.iloc[i + step - 1, target_column] for step in future_steps])

    # Convert lists to NumPy arrays after the loop
    features = np.array(features)  # Shape: (num_samples, n_past, num_features)
    response = np.array(response)  # Shape: (num_samples, len(future_steps))

    # Flatten the features to 2D array: (num_samples, n_past * num_features)
    features_flat = features.reshape(features.shape[0], -1)

    return features_flat, response


# Data loading
def data_loader(filepath):
    data = pd.read_csv(filepath)
    data['Time'] = pd.to_datetime(data['Time'],format='%Y-%m-%d %H:%M:%S')
    data.set_index('Time', inplace=True)

    return data

# Function to evaluate model predictions
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mbe = np.mean(y_pred - y_true)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, mape, mbe, rmse, r2


# Initialize CSV file and write headers
def initialize_csv(file_name):
    headers = ['lookback_window', 'features_used', 'learning_rate', 'degree', 'batch_size', 'epochs', 
               'MSE_1_day', 'MAE_1_day', 'MAPE_1_day', 'MBE_1_day', 'RMSE_1_day', 'R2_1_day',
               'MSE_3_day', 'MAE_3_day', 'MAPE_3_day', 'MBE_3_day', 'RMSE_3_day', 'R2_3_day',
               'MSE_5_day', 'MAE_5_day', 'MAPE_5_day', 'MBE_5_day', 'RMSE_5_day', 'R2_5_day']
    df = pd.DataFrame(columns=headers)
    df.to_csv(file_name, index=False)

# Append a single result (row) to the CSV file
def append_result_to_csv(file_name, result):
    df = pd.DataFrame([result])  # Convert result dictionary to DataFrame
    df.to_csv(file_name, mode='a', header=False, index=False)  # Append to CS

#                                    /* Feature combinations*/
def featuresComblist(features):
    import itertools

    initial_feature = ['Close'] # Starting with the closing price

    # Get all combinations of the features list and add to the initial feature (Closing Price)
    feature_combinations = []
    for i in range(len(features) + 1):
        for combination in itertools.combinations(features, i):
            feature_combinations.append(list(combination)+ initial_feature )
    
    return feature_combinations

######################################## Hyper-parameter tuninng #########################################


filepath= './Data/EURUSD_D1.csv'
dataset = data_loader(filepath)
multiVarData = multivariateFeatureEngineering(dataset) #Include addtional features outside of OHLCV
cols  = [col for col in multiVarData.columns if col!='Close'] + ['Close'] # Put target on End
multiVarData = multiVarData[cols]


# Parameters to optimise

features=['Open', 'High', 'Low', 'Volume', '50_sma', '200_sma', '50_ema',
       '100_ema', 'MACD_line', 'Signal_line', 'ADX', 'RSI', 'stoch_k',
       'stoch_d', 'ATR']
feature_combinations = featuresComblist(features)  # feature combinations
lookback_windows = [1, 3, 5, 7, 10, 15]
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [16, 32, 64, 128]
epochs = [10, 20, 50, 100]
polyn_degrees = [2, 3]

target_col=-1

future_steps = [1, 3, 5] 



########################################## Hyperparameter search ##########################################


def grid_Search():
    # Initialize CSV to store results
    file_name = 'PR_hyperparameter_tuning_results.csv'
    initialize_csv(file_name)

    features=['Open', 'High', 'Low', 'Volume', '50_sma', '200_sma', '50_ema',
        '100_ema', 'MACD_line', 'Signal_line', 'ADX', 'RSI', 'stoch_k',
        'stoch_d', 'ATR']
    feature_combinations = featuresComblist(features)  # feature combinations
    lookback_windows = [1, 3, 5, 7, 10, 15]
    learning_rates = [0.1, 0.01, 0.001]
    batch_sizes = [16, 32, 64, 128]
    epochs = [10, 20, 50, 100]
    polyn_degrees = [2, 3]


    # Load your data
    filepath = './Data/EURUSD_D1.csv' 
    dataset = data_loader(filepath)
    multiVarData = multivariateFeatureEngineering(dataset)
    cols  = [col for col in multiVarData.columns if col!='Close'] + ['Close'] # Put target on End
    multiVarData = multiVarData[cols]

    #Loop over all combinations of hyper-parameters
    for features_used in feature_combinations:

        multiVarData_subset = multiVarData[features_used] # select subset of data

        for lookback_window in lookback_windows:

            for lr in learning_rates:

                for degree in polyn_degrees:

                    for batch_size in batch_sizes:

                        for epoch in epochs:

                            future_steps = [1, 3, 5]  # model outputs

                            # Prepare features and response for the current hyperparameter combination
                            features, response = multivariateFeatureLagMultiStep(multiVarData_subset, lookback_window, future_steps, target_col)
                        
                             # Split data into training and testing sets
                            X_train, X_test, Y_train, Y_test = train_test_split(features, response, test_size=0.2, random_state=12, shuffle=False)

                            # Standardize features
                            scaler = StandardScaler()
                            X_train = scaler.fit_transform(X_train)
                            X_test = scaler.transform(X_test)

                            # Train the model with the current hyperparameter combination
                            model_pr = PolynomialRegressionMultiOutput(degree, lr, epoch, batch_size)
                            model_pr.fit(X_train, Y_train)

                            # Make predictions
                            predictions = model_pr.predict(X_test)
                            
                            #Error handling:
                            predictions = np.nan_to_num(predictions, nan=50)
                            #clip values to prevent overflow
                            predictions = np.clip(predictions, -50, 50)

                            # Evaluate the model for each forecast horizon
                            mse_1_day, mae_1_day, mape_1_day, mbe_1_day, rmse_1_day, r2_1_day = evaluate_model(Y_test[:, 0], predictions[:, 0])
                            mse_3_day, mae_3_day, mape_3_day, mbe_3_day, rmse_3_day, r2_3_day = evaluate_model(Y_test[:, 1], predictions[:, 1])
                            mse_5_day, mae_5_day, mape_5_day, mbe_5_day, rmse_5_day, r2_5_day = evaluate_model(Y_test[:, 2], predictions[:, 2])
                            #Store the results
                            result = {
                                'lookback_window': lookback_window,
                                'features_used': features_used,
                                'learning_rate': lr,
                                'degree': degree,
                                'batch_size': batch_size,
                                'epochs': epoch,
                                'MSE_1_day': mse_1_day,
                                'MAE_1_day': mae_1_day,
                                'MAPE_1_day': mape_1_day,
                                'MBE_1_day': mbe_1_day,
                                'RMSE_1_day': rmse_1_day,
                                'R2_1_day': r2_1_day,
                                'MSE_3_day': mse_3_day,
                                'MAE_3_day': mae_3_day,
                                'MAPE_3_day': mape_3_day,
                                'MBE_3_day': mbe_3_day,
                                'RMSE_3_day': rmse_3_day,
                                'R2_3_day': r2_3_day,
                                'MSE_5_day': mse_5_day,
                                'MAE_5_day': mae_5_day,
                                'MAPE_5_day': mape_5_day,
                                'MBE_5_day': mbe_5_day,
                                'RMSE_5_day': rmse_5_day,
                                'R2_5_day': r2_5_day
                            }

                            # Append the result to the CSV file
                            append_result_to_csv(file_name, result)

    print("Grid search completed and results saved to CSV!")



grid_Search()




/home/dgibrilly/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/home/dgibrilly/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/home/dgibrilly/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/home/dgibrilly/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
/home/dgibrilly/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:501: RuntimeWarning: overflow encountered in square

ValueError: Input contains NaN.